In [ ]:
import requests
import pandas as pd
import time
import os
import random
from datetime import datetime, timezone

# =========================
# 1) 설정
# =========================
# 본인 API KEY 넣기
API_KEY = ("")

SHARD = "steam"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/vnd.api+json",
}

SEED_NICKNAMES = ["PhilippineDealer"]

# (1) 날짜 필터: 고정 기준일 (UTC)
CUTOFF_UTC = datetime(2026, 2, 6, tzinfo=timezone.utc)

# (2) 매일 500개 단위로 새 파일 저장 (파일명 날짜 태그 자동)
RUN_TAG = datetime.now().strftime("%Y%m%d")  # 예: 20260210
OUTPUT_FILE = f"matches_steam_{RUN_TAG}.csv"
MATCHID_FILE = f"match_id_steam_{RUN_TAG}.csv"

# 중복 방지용(누적) matchId 로그 파일 (이건 계속 쌓이는 게 맞음)
ID_LOG_FILE = "steam_processed_match_ids.txt"

TARGET_COUNT = 500

# 안전한 대기(기본)
BASE_SLEEP_BETWEEN_PLAYER_CALLS = 1.5
BASE_SLEEP_BETWEEN_MATCH_CALLS = 0.4

# =========================
# 2) 중복 방지 로드/저장
# =========================
def load_processed_ids() -> set:
    if os.path.exists(ID_LOG_FILE):
        with open(ID_LOG_FILE, "r", encoding="utf-8") as f:
            return set(line.strip() for line in f if line.strip())
    return set()

def save_processed_id(match_id: str) -> None:
    with open(ID_LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"{match_id}\n")

# =========================
# 3) Session + 429/5xx 재시도
# =========================
sess = requests.Session()
sess.headers.update(HEADERS)

def get_with_retry(url: str, max_tries: int = 6, base_sleep: float = 1.0, timeout: int = 20):
    """
    - 200: 반환
    - 429: Retry-After 있으면 그만큼, 없으면 지수 백오프
    - 500~504: 지수 백오프
    - 그 외: 바로 반환(실패 처리)
    """
    last_res = None
    for i in range(max_tries):
        try:
            res = sess.get(url, timeout=timeout)
            last_res = res

            if res.status_code == 200:
                return res

            if res.status_code == 429:
                ra = res.headers.get("Retry-After")
                sleep_s = float(ra) if ra else (base_sleep * (2 ** i) + random.random())
                time.sleep(min(sleep_s, 60))
                continue

            if 500 <= res.status_code <= 504:
                time.sleep(base_sleep * (2 ** i) + random.random())
                continue

            return res
        except requests.RequestException:
            time.sleep(base_sleep * (2 ** i) + random.random())
            continue

    return last_res

# =========================
# 4) API 수집 함수
# =========================
def get_player_matches(player_name: str) -> list[str]:
    url = f"https://api.pubg.com/shards/{SHARD}/players?filter[playerNames]={player_name}"
    res = get_with_retry(url, max_tries=6, base_sleep=1.0, timeout=20)
    if res is None or res.status_code != 200:
        return []

    try:
        data = res.json()
        return [m["id"] for m in data["data"][0]["relationships"]["matches"]["data"]]
    except Exception:
        return []

def get_match_details(match_id: str):
    url = f"https://api.pubg.com/shards/{SHARD}/matches/{match_id}"
    res = get_with_retry(url, max_tries=6, base_sleep=1.0, timeout=25)
    if res is None or res.status_code != 200:
        return None, None

    try:
        data = res.json()
        m_attr = data["data"]["attributes"]

        # [필터 1] squad 모드만
        game_mode = m_attr.get("gameMode", "")
        if "squad" not in str(game_mode).lower():
            return None, None

        # [필터 2] 2026-02-06 00:00:00 UTC 이후만
        created_at = m_attr.get("createdAt")
        if not created_at:
            return None, None

        match_time = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
        if match_time < CUTOFF_UTC:
            return None, None

        match_info = {
            "matchId": match_id,
            "mapName": m_attr.get("mapName"),
            "gameMode": game_mode,
            "createdAt": created_at,
        }

        p_stats = []
        discovered = []

        for item in data.get("included", []):
            if item.get("type") == "participant":
                s = item.get("attributes", {}).get("stats", {})
                # 봇 제외
                if not str(s.get("playerId", "")).startswith("ai."):
                    p_stats.append({**match_info, **s})
                    name = s.get("name")
                    if name:
                        discovered.append(name)

        return p_stats, discovered

    except Exception:
        return None, None

# =========================
# 5) 메인 실행
# =========================
def main():
    processed_matches = load_processed_ids()
    player_queue = list(SEED_NICKNAMES)

    total_count = 0

    # (2) 오늘 파일은 새로 시작 (이미 있으면 덮어쓰기 원칙)
    # 아래 wrote_header 로직 때문에 굳이 미리 지울 필요는 없지만,
    # "오늘 실행은 항상 새 파일"이 목표면 지우는 게 더 명확함.
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)

    wrote_header = False
    collected_match_ids: list[str] = []

    print(f"--- 수집 시작 (목표: {TARGET_COUNT}개, cutoff(UTC): {CUTOFF_UTC.isoformat()}) ---")
    print(f"[SAVE] stats: {OUTPUT_FILE}")
    print(f"[SAVE] matchIds: {MATCHID_FILE}")
    print(f"[DEDUP LOG] {ID_LOG_FILE} (누적)")

    while player_queue and total_count < TARGET_COUNT:
        current_p = player_queue.pop(0)

        m_ids = get_player_matches(current_p)
        time.sleep(BASE_SLEEP_BETWEEN_PLAYER_CALLS)

        if not m_ids:
            continue

        for mid in m_ids:
            if total_count >= TARGET_COUNT:
                break

            # [중복 체크] 이미 처리된 matchId면 스킵
            if mid in processed_matches:
                continue

            m_data, new_players = get_match_details(mid)
            time.sleep(BASE_SLEEP_BETWEEN_MATCH_CALLS)

            if not m_data:
                # 필터 탈락/호출 실패 모두 여기로 옴
                processed_matches.add(mid)
                save_processed_id(mid)
                continue

            # 오늘 파일(OUTPUT_FILE)에만 append
            df = pd.DataFrame(m_data)
            df.to_csv(OUTPUT_FILE, mode="a", index=False, header=not wrote_header)
            wrote_header = True

            # matchId 목록 저장용 누적
            collected_match_ids.append(mid)

            # 중복 방지 기록 (누적)
            processed_matches.add(mid)
            save_processed_id(mid)

            total_count += 1

            # 가지 뻗기: 새로 발견된 유저 중 최대 5명 추가
            if new_players:
                player_queue.extend(new_players[:5])

            print(f"   [저장] {total_count}/{TARGET_COUNT}: {mid} (큐: {len(player_queue)})")

    # 매치ID만 별도 파일로 저장 (오늘 500개 단위 파일)
    pd.DataFrame({"matchId": collected_match_ids}).to_csv(MATCHID_FILE, index=False)

    print(f"--- 완료: 저장된 매치 {total_count}개 ---")
    print(f"[DONE] {OUTPUT_FILE}")
    print(f"[DONE] {MATCHID_FILE}")

if __name__ == "__main__":
    main()


--- 수집 시작 (목표: 500개, cutoff(UTC): 2026-02-06T00:00:00+00:00) ---
[SAVE] stats: matches_steam_20260211.csv
[SAVE] matchIds: match_id_steam_20260211.csv
[DEDUP LOG] steam_processed_match_ids.txt (누적)
   [저장] 1/500: 7413536f-674e-45fc-ad98-f56abd657f7f (큐: 5)
   [저장] 2/500: 9e1f1a93-8b7f-4119-bc0b-8c8560c32080 (큐: 10)
   [저장] 3/500: 0a5fc47e-c0e6-4870-a1f7-651ef62a10ef (큐: 15)
   [저장] 4/500: e7a4fea0-3f0e-4f09-b0af-a78afaa447d7 (큐: 20)
   [저장] 5/500: a8197f7a-7586-4e03-b5a7-f2f6e2685ba5 (큐: 25)
   [저장] 6/500: b3fdf2a2-9a28-4526-8e59-9447c12d9653 (큐: 30)
   [저장] 7/500: db7f1edc-6864-4590-817d-cf8a2597d288 (큐: 35)
   [저장] 8/500: 9e683b27-3ec4-41e0-9432-12c1738722fd (큐: 40)
   [저장] 9/500: 0b1903bb-93a5-4714-adcb-46387bf9b1c3 (큐: 45)
   [저장] 10/500: 19b2ca06-b8b0-4362-9fe3-28e9079e074e (큐: 50)
   [저장] 11/500: b5c9aab2-f2d9-4385-b991-1994c5bfd6f3 (큐: 55)
   [저장] 12/500: 26abf88b-d19a-41ef-8a36-95f430ff7532 (큐: 60)
   [저장] 13/500: 7cc58f78-3b7d-47f4-83c1-989a7f0896df (큐: 65)
   [저장] 14/500: 42d

In [ ]:
import pandas as pd
import requests
import duckdb
import os
import time
import glob
import random
from datetime import datetime

# =========================
# 설정
# =========================
API_KEY = ("")

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/vnd.api+json"
}

SHARD = "steam"

# 날짜 태그 자동
RUN_TAG = time.strftime("%Y%m%d")  # 예: 20260210
STATS_FILE = f"matches_steam_{RUN_TAG}.csv"

TEMP_DIR = "temp_parquet_files_steam"  # 개별 매치를 저장할 임시 폴더

MAP_NAMES = {
    "Baltic_Main": "Erangel",
    "Desert_Main": "Miramar",
    "Savage_Main": "Sanhok",
    "Dihorotok_Main": "Vikendi",
    "Tiger_Main": "Taego",
    "Neon_Main": "Rondo",
}

# =========================
# 폴더 준비
# =========================
if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)

# =========================
# flatten 로직
# =========================
def flatten_log(log: dict, match_id: str) -> dict:
    row = {"matchId": match_id}
    for key, value in log.items():
        if key in ["_D", "_T", "elapsedTime"]:
            row[key] = value
        elif isinstance(value, dict):
            for sub_key, sub_value in value.items():
                if isinstance(sub_value, dict):
                    for s_key, s_value in sub_value.items():
                        row[f"{key}_{sub_key}_{s_key}"] = s_value
                else:
                    row[f"{key}_{sub_key}"] = sub_value
        else:
            row[key] = value
    return row

# =========================
# Session + timeout
# =========================
sess = requests.Session()
sess.headers.update(HEADERS)

# =========================
# 429/5xx 재시도 함수
# =========================
def get_with_retry(sess: requests.Session, url: str, max_tries: int = 6, base_sleep: float = 1.0, timeout: int = 20):
    """
    - 200이면 바로 반환
    - 429면 Retry-After 있으면 그만큼, 없으면 지수 백오프 + 랜덤
    - 500~504도 지수 백오프 재시도
    - 그 외 코드는 그대로 반환(실패 처리)
    """
    last_res = None
    for i in range(max_tries):
        try:
            res = sess.get(url, timeout=timeout)
            last_res = res

            if res.status_code == 200:
                return res

            if res.status_code == 429:
                ra = res.headers.get("Retry-After")
                sleep_s = float(ra) if ra else (base_sleep * (2 ** i) + random.random())
                time.sleep(min(sleep_s, 60))
                continue

            if 500 <= res.status_code <= 504:
                time.sleep(base_sleep * (2 ** i) + random.random())
                continue

            return res  # 그 외는 재시도하지 않음

        except requests.RequestException:
            # 네트워크 에러도 백오프로 재시도
            time.sleep(base_sleep * (2 ** i) + random.random())
            continue

    return last_res

# =========================
# 텔레메트리도 timeout + 재시도
# =========================
def get_telemetry(t_url: str, match_id: str, timeout: int = 30, max_tries: int = 5, base_sleep: float = 1.0):
    """
    텔레메트리 URL은 보통 퍼블릭이라 헤더 없이도 되지만,
    timeout/재시도는 넣어두는 게 안정적입니다.
    """
    last_res = None
    for i in range(max_tries):
        try:
            res = requests.get(t_url, timeout=timeout)
            last_res = res

            if res.status_code == 200:
                logs = res.json()
                return [flatten_log(log, match_id) for log in logs]

            if res.status_code == 429:
                ra = res.headers.get("Retry-After")
                sleep_s = float(ra) if ra else (base_sleep * (2 ** i) + random.random())
                time.sleep(min(sleep_s, 60))
                continue

            if 500 <= res.status_code <= 504:
                time.sleep(base_sleep * (2 ** i) + random.random())
                continue

            return []

        except Exception:
            time.sleep(base_sleep * (2 ** i) + random.random())
            continue

    return []

# =========================
# 메인 수집
# =========================
def run_collection():
    if not os.path.exists(STATS_FILE):
        print(f"[ERROR] 파일을 찾을 수 없습니다: {STATS_FILE}")
        return

    df_stats = pd.read_csv(STATS_FILE)

    if "gameMode" in df_stats.columns:
        df_squad = df_stats[df_stats["gameMode"].str.contains("squad", case=False, na=False)].copy()
    else:
        df_squad = df_stats.copy()

    match_count = len(df_squad["matchId"].unique()) if "matchId" in df_squad.columns else 0
    print(f"[INFO] 필터링 완료: 스쿼드 매치 총 {match_count}개")

    for map_code, map_alias in MAP_NAMES.items():
        if "mapName" not in df_squad.columns or "matchId" not in df_squad.columns:
            print("[ERROR] STATS_FILE에 mapName/matchId 컬럼이 필요합니다.")
            return

        matches = df_squad[df_squad["mapName"] == map_code]["matchId"].unique().tolist()
        if not matches:
            continue

        print(f"{map_alias} 수집 시작. (대상: {len(matches)}개)")

        for mid in matches:
            # 개별 매치 파일 경로 (예: temp_parquet_files/Erangel_<matchId>.parquet)
            match_save_path = os.path.join(TEMP_DIR, f"{map_alias}_{mid}.parquet")

            # 이미 저장된 매치라면 스킵 (재시작 시 유용)
            if os.path.exists(match_save_path):
                continue

            match_url = f"https://api.pubg.com/shards/{SHARD}/matches/{mid}"
            res = get_with_retry(sess, match_url, max_tries=6, base_sleep=1.0, timeout=20)

            if (res is None) or (res.status_code != 200):
                status = res.status_code if res is not None else "NO_RESPONSE"
                print(f"{mid} 호출 실패 (Status: {status})")
                continue

            try:
                assets = res.json().get("included", [])
                t_url = next(
                    (item.get("attributes", {}).get("URL") for item in assets if item.get("type") == "asset"),
                    None
                )

                if not t_url:
                    print(f"{mid} telemetry URL 없음")
                    continue

                data = get_telemetry(t_url, mid, timeout=30, max_tries=5, base_sleep=1.0)
                if not data:
                    print(f"{mid} telemetry 비어있음/실패")
                    continue

                df = pd.DataFrame(data)
                df.to_parquet(match_save_path, index=False, compression="snappy")
                print(f"    + [저장완료] {map_alias} ID: {mid} ({len(data)} rows)")

                # 너무 빡세게 치지 않도록 간격
                time.sleep(1.2)

            except Exception as e:
                print(f"{mid} 처리 중 오류: {e}")

    merge_all_to_final()

def merge_all_to_final():
    print("모든 수집 완료. 최종 병합 작업 시작")

    # 출력 폴더(선택)
    OUT_DIR = "final_telemetry"
    os.makedirs(OUT_DIR, exist_ok=True)

    for map_alias in MAP_NAMES.values():
        # glob 패턴(duckdb가 이 패턴을 그대로 읽을 수 있음)
        pattern = os.path.join(TEMP_DIR, f"{map_alias}_*.parquet")
        map_files = glob.glob(pattern)
        if not map_files:
            continue

        print(f"{map_alias} 병합 중. ({len(map_files)}개 파일)")

        final_filename = os.path.join(OUT_DIR, f"{map_alias}_telemetry_steam_{RUN_TAG}.parquet")

        # 핵심: pandas concat 대신 duckdb가 바로 parquet로 써줌 (RAM 절약)
        duckdb.sql(f"""
            COPY (
                SELECT * FROM read_parquet('{pattern}', union_by_name=True)
            ) TO '{final_filename}'
            (FORMAT PARQUET, COMPRESSION 'snappy');
        """)

        print(f"{final_filename} 생성 완료")

if __name__ == "__main__":
    run_collection()


[INFO] 필터링 완료: 스쿼드 매치 총 500개
Erangel 수집 시작. (대상: 130개)
Miramar 수집 시작. (대상: 124개)
Taego 수집 시작. (대상: 124개)
Rondo 수집 시작. (대상: 122개)
모든 수집 완료! 최종 병합 작업을 시작합니다
Erangel 병합 중... (130개 파일)
final_telemetry\Erangel_telemetry_steam_20260211.parquet 생성 완료
Miramar 병합 중... (124개 파일)
final_telemetry\Miramar_telemetry_steam_20260211.parquet 생성 완료
Taego 병합 중... (124개 파일)
final_telemetry\Taego_telemetry_steam_20260211.parquet 생성 완료
Rondo 병합 중... (122개 파일)
final_telemetry\Rondo_telemetry_steam_20260211.parquet 생성 완료
